# 1. Blocker Fraud Company
 <font size="2"> Dados e problema negócio hipotéticos</font>
 
A Blocker Fraud Company é uma empresa especializada na detecção de fraudes em transações financeiras feitas atravś de dispositivos móveis.

O modelo de negócio da empresa é do tipo Serviço com a monetização feita por performance do serviço prestado, ou seja, o usuário paga uma taxa fixa sobre o sucesso da detecção de fraude das transações.

## 1.1. Estratégia de expansão

A Blocker Fraud Company está em fase de expansão no Brasil e para adquirir clientes mais rapidamente, ela adotou uma estratégia muito agressiva. A estratégia funciona da seguinte forma:

1. A empresa vai receber 25% do valor de cada transação detectada verdadeiramente como fraude.
2. A empresa vai receber 5% do valor de cada transação detectada como fraude, porém a transação é verdadeiramente legítima.
3. A empresa vai devolver 100% do valor para o cliente, a cada transação detectada como legítima, porém a transação é verdadeiramente uma fraude.

Com essa estratégia agressiva a empresa assume os riscos em falhar na detecção de fraude e é remunerada na detecção assertiva das fraudes.

## 1.2. Objetivo

Criar um modelo de alta precisão e acurácia na detecção de fraudes de transações feitas através de dispositivos móveis.

### 1.2.1. Entregáveis

Modelo em produção no qual seu acesso será feito via API, ou seja, os clientes enviarão suas transações via API para que o seu modelo as classifique como fraudulentas ou legítimas.

Além disso, você precisará entregar um relatório reportando a performance e os resultados do seu modelo em relação ao lucro e prejuízo que a empresa terá ao usar o modelo que você produziu. No seu relatório deve conter as respostas para as seguintes perguntas:

1. Qual a Precisão e Acurácia do modelo?
2. Qual a Confiabilidade do modelo em classificar as transações como legítimas ou fraudulentas?
3. Qual o Faturamento Esperado pela Empresa se classificarmos 100% das transações com o modelo?
4. Qual o Prejuízo Esperado pela Empresa em caso de falha do modelo?
5. Qual o Lucro Esperado pela Blocker Fraud Company ao utilizar o modelo?

# 2. Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

#display
pd.set_option('display.float_format', lambda x: '%.5f' % x)

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

from sklearn import model_selection
from sklearn import ensemble
from sklearn import tree
from sklearn import linear_model
from sklearn import pipeline
from sklearn import preprocessing

from scipy import stats


from feature_engine import wrappers 
from feature_engine import selection
from feature_engine import imputation
from feature_engine import encoding
from feature_engine import discretisation

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,\
confusion_matrix, matthews_corrcoef, make_scorer, roc_curve, precision_recall_curve, classification_report

## 2.1. Utils functions

In [2]:
def performance(model, x_train, y_train):
    
    scoring = {'accuracy': 'accuracy',
               'precision': make_scorer(precision_score),
               'recall': make_scorer(recall_score),
               'f1_score': make_scorer(f1_score),
               'roc_auc_score': make_scorer(roc_auc_score),
               'mcc': make_scorer(matthews_corrcoef)}

    scores = model_selection.cross_validate(model, x_train, y_train, cv=5, scoring=scoring)

    performance = pd.DataFrame.from_dict(scores).drop(['fit_time', 'score_time'], axis=1)
    performance = pd.DataFrame(performance.mean()).T
    return performance

# 3. Data

## 3.1. Load dataset

In [3]:
df = pd.read_parquet('../data/raw/df_applied_features.parquet.gzip') 

In [4]:
df_test = pd.read_parquet('../data/raw/df_applied_features_test.parquet.gzip') 

# 4. Model

## 4.1. Define feature and target

In [5]:
features = df.drop('isFraud', axis=1).columns.tolist()
target = 'isFraud'

## 4.2. Split data into train, valid and test

In [6]:
X = df[features]
y = df[target].values

X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X,
                                                                      y,
                                                                      random_state=42,
                                                                      test_size=0.3,
                                                                      stratify=y)

In [7]:
cols_to_include = ['isFlaggedFraud', 'is_high_amount', 'is_M_customer', 
                   'is_new_balance_orig_zero', 'is_orig_old_balance_greater_new',
                   'is_new_balance_dest_zero', 'is_dest_old_balance_greater_new']

columns_drop = ['nameOrig', 'nameDest']

for col in cols_to_include:
    X_train[col] = X_train[col].astype('category')
    X_valid[col] = X_valid[col].astype('category')
    
X_train.drop(columns_drop, axis=1, inplace=True)
X_valid.drop(columns_drop, axis=1, inplace=True)

In [8]:
X_test = df_test[features].copy()
y_test = df_test[target]


for col in cols_to_include:
    X_test[col] = X_test[col].astype('category')
    
X_test.drop(columns_drop, axis=1, inplace=True)

## 4.3. Pipeline

In [9]:
cat_features = ['is_new_balance_dest_zero',
                'type',
                'is_orig_old_balance_greater_new',
                'is_M_customer',
                'is_high_amount',
                'is_new_balance_orig_zero',
                'is_dest_old_balance_greater_new',
                'isFlaggedFraud']
num_features = ['amount',
                'oldbalanceOrg',
                'newbalanceOrig',
                'oldbalanceDest',
                'newbalanceDest',
                'diff_orig',
                'diff_dest']

In [10]:
rf_clf = ensemble.RandomForestClassifier(n_jobs = -1, random_state=42)

In [11]:
cat_enc = encoding.CountFrequencyEncoder(variables=cat_features)
rare_enc = encoding.RareLabelEncoder(variables=['type', 'isFlaggedFraud'])
numerical_enc = wrappers.SklearnTransformerWrapper(transformer = preprocessing.StandardScaler(),
                                                   variables = num_features)

In [12]:
pipe = pipeline.Pipeline(steps = [("enconding_rare", rare_enc),
                                  ("enconding_cat", cat_enc),
                                  ("enconding_num", numerical_enc),
                                  ("model", rf_clf)])

# 5. Hyperparameter tuning

In [13]:
param_grid = [{  "model": [ensemble.RandomForestClassifier()],
                 "model__n_estimators": [10, 100, 1000],
                 "model__max_depth":[5,8,15,25,30,None],
                 "model__min_samples_leaf":[1,2,5,10,15,100],
                 "model__max_leaf_nodes": [2, 5,10]}]

In [14]:
# set up the search
search = model_selection.RandomizedSearchCV(pipe,
                            param_grid,
                            scoring='recall',
                            cv=3,
                            n_iter = 10,
                            random_state=42,
                            n_jobs=4,
                            verbose=1000)

# find best hyperparameters
search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/home/matheus/anaconda3/envs/blocker_fraud/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning: The number of unique categories for variable type is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/home/matheus/anaconda3/envs/blocker_fraud/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning: The number of unique categories for variable isFlaggedFraud is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/home/matheus/anaconda3/envs/blocker_fraud/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning: The number of unique categories for variable type is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/home/matheus/anaconda3/envs/blocker_fraud/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning:

/home/matheus/anaconda3/envs/blocker_fraud/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning: The number of unique categories for variable type is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/home/matheus/anaconda3/envs/blocker_fraud/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning: The number of unique categories for variable isFlaggedFraud is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/home/matheus/anaconda3/envs/blocker_fraud/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning: The number of unique categories for variable type is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/home/matheus/anaconda3/envs/blocker_fraud/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning:

/home/matheus/anaconda3/envs/blocker_fraud/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning: The number of unique categories for variable type is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/home/matheus/anaconda3/envs/blocker_fraud/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning: The number of unique categories for variable isFlaggedFraud is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/home/matheus/anaconda3/envs/blocker_fraud/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning: The number of unique categories for variable type is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/home/matheus/anaconda3/envs/blocker_fraud/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:161: UserWarning:

[CV 1/3; 2/10] START model=RandomForestClassifier(), model__max_depth=15, model__max_leaf_nodes=2, model__min_samples_leaf=1, model__n_estimators=10
[CV 1/3; 2/10] END model=RandomForestClassifier(), model__max_depth=15, model__max_leaf_nodes=2, model__min_samples_leaf=1, model__n_estimators=10;, score=0.000 total time=  24.4s
[CV 2/3; 2/10] START model=RandomForestClassifier(), model__max_depth=15, model__max_leaf_nodes=2, model__min_samples_leaf=1, model__n_estimators=10
[CV 2/3; 2/10] END model=RandomForestClassifier(), model__max_depth=15, model__max_leaf_nodes=2, model__min_samples_leaf=1, model__n_estimators=10;, score=0.000 total time=  26.2s
[CV 3/3; 3/10] START model=RandomForestClassifier(), model__max_depth=15, model__max_leaf_nodes=5, model__min_samples_leaf=10, model__n_estimators=1000
[CV 3/3; 3/10] END model=RandomForestClassifier(), model__max_depth=15, model__max_leaf_nodes=5, model__min_samples_leaf=10, model__n_estimators=1000;, score=0.349 total time=34.0min
[CV 1/3

[CV 3/3; 1/10] START model=RandomForestClassifier(), model__max_depth=15, model__max_leaf_nodes=5, model__min_samples_leaf=5, model__n_estimators=10
[CV 3/3; 1/10] END model=RandomForestClassifier(), model__max_depth=15, model__max_leaf_nodes=5, model__min_samples_leaf=5, model__n_estimators=10;, score=0.264 total time=  31.9s
[CV 1/3; 3/10] START model=RandomForestClassifier(), model__max_depth=15, model__max_leaf_nodes=5, model__min_samples_leaf=10, model__n_estimators=1000
[CV 1/3; 3/10] END model=RandomForestClassifier(), model__max_depth=15, model__max_leaf_nodes=5, model__min_samples_leaf=10, model__n_estimators=1000;, score=0.354 total time=34.2min
[CV 3/3; 9/10] START model=RandomForestClassifier(), model__max_depth=30, model__max_leaf_nodes=10, model__min_samples_leaf=15, model__n_estimators=1000
[CV 3/3; 9/10] END model=RandomForestClassifier(), model__max_depth=30, model__max_leaf_nodes=10, model__min_samples_leaf=15, model__n_estimators=1000;, score=0.427 total time=36.4min

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('enconding_rare',
                                              RareLabelEncoder(variables=['type',
                                                                          'isFlaggedFraud'])),
                                             ('enconding_cat',
                                              CountFrequencyEncoder(variables=['is_new_balance_dest_zero',
                                                                               'type',
                                                                               'is_orig_old_balance_greater_new',
                                                                               'is_M_customer',
                                                                               'is_high_amount',
                                                                               'is_new_balance_orig_zero',
                                                               

## 5.1. Best parameters for model

In [15]:
search.best_params_

{'model__n_estimators': 1000,
 'model__min_samples_leaf': 15,
 'model__max_leaf_nodes': 10,
 'model__max_depth': 30,
 'model': RandomForestClassifier(max_depth=30, max_leaf_nodes=10, min_samples_leaf=15,
                        n_estimators=1000)}

## 5.2. Training model with best parameters

In [16]:
rf_clf_tuned = ensemble.RandomForestClassifier(n_jobs = -1, 
                                               random_state=42,
                                               max_depth=30, 
                                               max_leaf_nodes=10, 
                                               min_samples_leaf=15,
                                               n_estimators=1000)
pipe_tuned = pipeline.Pipeline(steps = [("enconding_rare", rare_enc),
                                  ("enconding_cat", cat_enc),
                                  ("enconding_num", numerical_enc),
                                  ("model", rf_clf_tuned)])

In [17]:
pipe_tuned.fit(X_train, y_train)

Pipeline(steps=[('enconding_rare',
                 RareLabelEncoder(variables=['type', 'isFlaggedFraud'])),
                ('enconding_cat',
                 CountFrequencyEncoder(variables=['is_new_balance_dest_zero',
                                                  'type',
                                                  'is_orig_old_balance_greater_new',
                                                  'is_M_customer',
                                                  'is_high_amount',
                                                  'is_new_balance_orig_zero',
                                                  'is_dest_old_balance_greater_new',
                                                  'isFlaggedFraud'])),
                ('enconding_num',
                 SklearnTransformerWrapper(transformer=StandardScaler(),
                                           variables=['amount', 'oldbalanceOrg',
                                                      'newbalanceOrig',
       

## 5.3. Saving new model

In [18]:
joblib.dump(pipe_tuned, '../models/model_tuned_cicle_1.pkl', compress = 1)

['../models/model_tuned_cicle_1.pkl']

## 5.4. Performance training dataset

In [19]:
performance(pipe_tuned, X_train, y_train)

,test_accuracy,test_precision,test_recall,test_f1_score,test_roc_auc_score,test_mcc
0,0.99925,0.99874,0.42272,0.59388,0.71136,0.64943


## 5.5. Performance testing dataset

In [20]:
performance(pipe_tuned, X_test, y_test)

,test_accuracy,test_precision,test_recall,test_f1_score,test_roc_auc_score,test_mcc
0,0.99914,0.96000,0.38366,0.54157,0.69182,0.60209


# 6. Conclusion

Podemos ver que a forma escolhida para o tunning não encontrou os melhores parâmetros para aumentar o recall, dado que o nos dados de teste o modelo não conseguiu performar satisfatóriamente, isso pode indicar que faltou mais parâmetros ou mais tempo para que ele encontre os melhores parâmetros e também podemos testar futuramente outras formas de encontrar os parâmetros.

Lembrando que também podemos utilizar o tunning nas etapas de feature engineering, como por exemplo, utilizar outras formas de transformar os dados categóricos e númericos.

Portanto, vamos manter o modelo anteriormente selecionado como o modelo utilizado nesse primeiro ciclo.